In [2]:
!pip install sentence-transformers

In [2]:
import sqlite3
import pandas as pd
import zipfile
import io
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\poppo\anaconda3\envs\datascience\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Step 1: Ingesting Documents
def read_database(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    return conn, tables

In [3]:
import sqlite3
import pandas as pd
import zipfile
import io
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Ingesting Documents
def read_database(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    return conn, tables

def decode_and_extract(binary_data):
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    return subtitle_content.decode('latin-1')

def ingest_documents(database_path, sample_percentage=None):
    conn, tables = read_database(database_path)
    selected_table = tables[0][0]  # Assuming the first table is the one we want
    df = pd.read_sql_query(f"SELECT * FROM {selected_table}", conn)
    if sample_percentage:
        df = df.sample(frac=sample_percentage, random_state=42)
    df['file_content'] = df['content'].apply(decode_and_extract)
    return df

# Step 2: Cleaning and Preprocessing
def clean_text(text):
    # Example cleaning: remove time-stamps
    cleaned_text = text  # Placeholder for actual cleaning steps
    return cleaned_text

def preprocess_documents(documents):
    cleaned_documents = documents.apply(clean_text)
    return cleaned_documents

# Step 3: Generating Text Vectors
def generate_bow_vectors(documents):
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform(documents)
    return bow_matrix

def generate_tfidf_vectors(documents):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
    return tfidf_matrix

def generate_sentence_embeddings(documents):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    embeddings = model.encode(documents)
    return embeddings

# Step 4: Document Chunking
def chunk_document(document, chunk_size=500, overlap_size=50):
    chunks = []
    words = document.split()
    for i in range(0, len(words), chunk_size - overlap_size):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# Step 5: Storing Embeddings
def store_embeddings(embeddings, database_path):
    # Placeholder for storing embeddings in ChromaDB or any other database
    pass

# Step 6: Retrieving Documents
def preprocess_query(query):
    # Placeholder for query preprocessing steps
    return query

def vectorize_query(query):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    query_embedding = model.encode([query])
    return query_embedding

def calculate_similarity(query_embedding, document_embeddings):
    similarities = cosine_similarity(query_embedding, document_embeddings)
    return similarities[0]

def retrieve_documents(df, similarities, top_n=5):
    df['similarity'] = similarities
    df = df.sort_values(by='similarity', ascending=False)
    return df.head(top_n)

# Main Function
def main():
    database_path = r"C:\Users\poppo\Desktop\innomatics research labs\innomatics data science intern\tasks\PART4\TEAM PROEJCT\data-20240407T090330Z-001\data\eng_subtitles_database.db"
    
    # Step 1: Ingesting Documents
    df = ingest_documents(database_path, sample_percentage=0.3)
    
    # Step 2: Cleaning and Preprocessing
    cleaned_documents = preprocess_documents(df['file_content'])
    
    # Step 3: Generating Text Vectors
    bow_matrix = generate_bow_vectors(cleaned_documents)
    tfidf_matrix = generate_tfidf_vectors(cleaned_documents)
    sentence_embeddings = generate_sentence_embeddings(cleaned_documents)
    
    # Step 4: Document Chunking
    chunks = df['file_content'].apply(chunk_document)
    
    # Step 5: Storing Embeddings
    store_embeddings(sentence_embeddings, database_path)
    
    # Step 6: Retrieving Documents
    query = input(" actionmovies: ")
    preprocessed_query = preprocess_query(query)
    query_embedding = vectorize_query(preprocessed_query)
    
    similarities = calculate_similarity(query_embedding, sentence_embeddings)
    relevant_documents = retrieve_documents(df, similarities)
    print(relevant_documents)

if __name__ == "__main__":
    main()


c:\Users\poppo\anaconda3\envs\datascience\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\poppo\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 